# 数据处理
## 1. 爬虫部分

### 1.1 计算爬取的总数据量（条）

In [1]:
import pandas as pd

filepath = "DataCrawler/MyData/Data_.csv"
tot_len = 0
for i in range(1, 29):
    pathl = filepath[0:-4]
    pathr = filepath[-4:]
    nowpath = pathl + str(i) + pathr
    # print(nowpath)
    data = pd.read_csv(nowpath)
    tot_len += len(data)
    # print(data)
print(tot_len)

120900


### 1.2 查看一个文件的数据

In [2]:
filepath = "DataCrawler/MyData/Data_1.csv"
data = pd.read_csv(filepath)
print(data)

       idiom1   idiom2                                           sentence
0        哀兵必胜     骄兵必败  有句话叫做骄兵必败，哀兵必胜。虽然我们队赢了这场比赛，但我们不能骄傲，否则下次输的一方一定是我们。
1        哀兵必胜  置之死地而后生                     哀兵必胜，置之死地而后生，更何况他们之间本身就不是生死之斗。
2        哀兵必胜     置之死地                     哀兵必胜，置之死地而后生，更何况他们之间本身就不是生死之斗。
3     置之死地而后生     置之死地                     哀兵必胜，置之死地而后生，更何况他们之间本身就不是生死之斗。
4        哀兵必胜     屡战屡败  因为牺牲的都是身边的亲友，感情很深，正和哀兵必胜的兵家奥义，可是碰上太平军就是屡战屡败，只能...
...       ...      ...                                                ...
7107     闭门思过     一败涂地            孰不料玫瑰园一役，一败涂地一手育大的利达行亦拱手让人，只得蛰伏广州，闭门思过。
7108     闭门思过     拱手让人            孰不料玫瑰园一役，一败涂地一手育大的利达行亦拱手让人，只得蛰伏广州，闭门思过。
7109     一败涂地     拱手让人            孰不料玫瑰园一役，一败涂地一手育大的利达行亦拱手让人，只得蛰伏广州，闭门思过。
7110     闭门思过     坐以待毙  在当今主流被各种门事件牵着走的时候，俺，也不坐以待毙，咱也来个门，啥门？闭门！——闭门思过、...
7111     闭门思过     得过且过  【光棍节的种过法】快乐法：一笑而过；认命法：得过且过；潇洒法：飘过；实诚法：路过；忏悔法：闭...

[7112 rows x 3 columns]


## 2. 打标签部分
### 2.1 将csv文件转成xlsx文件

In [5]:
import pandas as pd
def csv_to_xlsx_pd(fileFrom, fileTo):
    csv = pd.read_csv('DataCrawler/MyData/' + fileFrom, encoding='utf-8')
    csv.to_excel('DataCrawler/MyXlsx/' + fileTo, sheet_name='data', index = False)

for i in range(1, 29):
    fileFrom = "Data_" + str(i) + ".csv"
    fileTo = "Data_" + str(i) + ".xlsx"
    csv_to_xlsx_pd(fileFrom, fileTo)

——**漫长的打标签**——

## 3. 添加成语的举例`example`以及解释`explanation`数据字段
### 3.1 重构json文件的数据

In [1]:
import json

idiomDict = {}
# 读取idiom.json中的数据 重构
with open('idiom.json', 'r',encoding="utf-8") as f:
    list = f.readlines()[0]
    preData = json.loads(list)
    for data in preData:
        word = data["word"]
        idiomDict[word] = {}
        idiomDict[word]["explanation"] = data["explanation"]
        idiomDict[word]["example"] = data["example"]
# print(idiomDict) ############这里不要输出 否则JupyterNotebook会爆内存

### 3.2 添加字段并分割数据集

In [2]:
import pandas as pd
import numpy as np  
import re

# 保存为csv文件
def saveCsvData(datalist):
    if len(datalist) == 0:
        return
    data = pd.DataFrame(datalist)
    savepath = "IdiomData.csv"
    data.to_csv(savepath, index=False,
                header=['idiom1', 'idiom2', 'label', 'explanation1', 'example1', 'explanation2', 'example2'])

# 获取两个成语的解释和举例
def getExplanationAndExample(idiom1, idiom2):
    dic1 = idiomDict.get(idiom1)
    dic2 = idiomDict.get(idiom2)
    if dic1 == None or dic2 == None:               # 在idiomDict中查找 若没有 则跳过
        return (None, None, None, None)
    example1 = dic1['example']
    example2 = dic2['example']
    example1 = re.sub('(～)+', idiom_i[0], example1) # 使用正则表达式 将成语加入到造句中
    example2 = re.sub('(～)+', idiom_i[1], example2)
    example1 = re.sub('(★.*)+', "", example1)       # 使用正则表达式 去除造句来源
    example2 = re.sub('(★.*)+', "", example2)
    
    explanation1 = dic1['explanation']
    explanation2 = dic2['explanation']
    return (explanation1, example1, explanation2, example2)
    
allData = []

# 遍历所有Xlsx文件 从idiomDict中添加字段
for page in range(1, 2):
    file_path = "Data_" + str(page) + ".xlsx"
    data_xlsx = pd.read_excel(file_path)
    # print(data_xlsx)
    data_list = np.array(data_xlsx)

    for data_i in data_list:
        idiom_i = [data_i[0], data_i[1], data_i[3]]
        explanation1, example1, explanation2, example2 = getExplanationAndExample(data_i[0], data_i[1])
        if explanation1 == None:
            continue
        idiom_i.append(explanation1)
        idiom_i.append(example1)
        idiom_i.append(explanation2)
        idiom_i.append(example2)
        
        allData.append(idiom_i)

# 添加若干不具有逻辑关系的成语 与打标签数据个数相同
# for i in range(len(allData)):
#     list(idiomDict.keys())

# print(allData)
# 保存数据
savepath = "IdiomData_train.csv"
saveCsvData(allData)
print(len(allData))
# savepath = "IdiomData_test.csv"
# saveCsvData(allData)

<class 'dict'>
173


还需要的操作：
* 随机挑选两个成语 作为不具有逻辑关系的例子 加入到数据中
* 按照一定比例分开 train 和 test数据集

In [17]:
filepath = "IdiomData.csv"
data = pd.read_csv(filepath)
print(data)

    idiom1 idiom2  label                                 explanation1  \
0     哀兵必胜   骄兵必败      2  原意是力量相当的两军对阵，悲愤的一方获得胜利◇指因受欺侮而奋起抵抗的军队，必定能取胜。   
1     哀兵必胜   屡战屡败      2  原意是力量相当的两军对阵，悲愤的一方获得胜利◇指因受欺侮而奋起抵抗的军队，必定能取胜。   
2     哀兵必胜   穷寇莫追      1  原意是力量相当的两军对阵，悲愤的一方获得胜利◇指因受欺侮而奋起抵抗的军队，必定能取胜。   
3     哀兵必胜   破釜沉舟      1  原意是力量相当的两军对阵，悲愤的一方获得胜利◇指因受欺侮而奋起抵抗的军队，必定能取胜。   
4     哀鸿遍野   疮痍满目      1  哀鸿哀鸣的鸿雁。比喻啼饥号寒的灾民。比喻在天灾人祸中到处都是流离失所、呻吟呼号的饥民。   
..     ...    ...    ...                                          ...   
168   天长地久   海枯石烂      1           跟天和地存在的时间那样长。形容时间悠久。也形容永远不变（多指爱情）。   
169   天长地久   相濡以沫      1           跟天和地存在的时间那样长。形容时间悠久。也形容永远不变（多指爱情）。   
170   甜言蜜语   海誓山盟      1                      象蜜糖一样甜的话。比喻为了骗人而说得动听的话。   
171   白头偕老   心心相印      1                       白头头发白；偕共同。夫妻相亲相爱，一直到老。   
172   将心比心   心心相印      1                                  设身处地地为别人着想。   

                                  example1  \
0            不，哀兵必胜！不要乐，要哀，何小姐能弹《易水吟》的谱子吗？   
1            不，哀兵必胜！不要乐，要哀，何小姐能

**至此，数据集构建完成**